In [1]:
!pip install openmeteo_requests
!pip install requests_cache
!pip install retry_requests


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [5]:
import json
with open('G:/IISc Bangalore/IUDX/traffic_violations1.json', 'r') as f:
    data = json.load(f)
traffic_df = pd.DataFrame(data)
traffic_df = pd.json_normalize(data)
traffic_df.head(5)

,id,vehicleType,alertType,observationDateTime,cameraUsage,junctionName,location.type,location.coordinates
0,5b778007-797e-4313-b34c-c7c4812365bb,Car,STOP_LINE_VIOLATION,2024-02-21T13:29:49+05:30,ANPR,SampigeRoad,Point,"[77.571138, 13.008491]"
1,5b778007-797e-4313-b34c-c7c4812365bb,Car,LICENCE_PLATE_RECOGNITION,2024-02-21T13:30:00+05:30,ANPR,VanivilasJN,Point,"[77.576809, 12.948272]"
2,5b778007-797e-4313-b34c-c7c4812365bb,Motorbike,STOP_LINE_VIOLATION,2024-02-21T13:30:00+05:30,ANPR,Mariyappacircle,Point,"[77.63721, 13.022783]"
3,5b778007-797e-4313-b34c-c7c4812365bb,Motorbike,RED_LIGHT_VIOLATION_DETECTION,2024-02-21T13:29:59+05:30,ANPR,Mariyappacircle,Point,"[77.63721, 13.022783]"
4,5b778007-797e-4313-b34c-c7c4812365bb,Motorbike,RED_LIGHT_VIOLATION_DETECTION,2024-02-21T13:10:36+05:30,ANPR,Chindrikahotel,Point,"[77.593048, 12.989344]"


In [6]:
traffic_df.tail(5)

,id,vehicleType,alertType,observationDateTime,cameraUsage,junctionName,location.type,location.coordinates
4753001,5b778007-797e-4313-b34c-c7c4812365bb,Car,RED_LIGHT_VIOLATION_DETECTION,2024-03-20T22:29:44+05:30,ANPR,SwamiVivekananda,Point,"[77.631421, 12.981592]"
4753002,5b778007-797e-4313-b34c-c7c4812365bb,Motorbike,STOP_LINE_VIOLATION,2024-03-20T22:29:43+05:30,ANPR,JPnagar-8thmain-9thcross,Point,"[77.595183, 12.910289]"
4753003,5b778007-797e-4313-b34c-c7c4812365bb,Motorbike,RED_LIGHT_VIOLATION_DETECTION,2024-03-20T22:29:42+05:30,ANPR,AXASignal,Point,"[77.615204, 12.91602]"
4753004,5b778007-797e-4313-b34c-c7c4812365bb,Undetermined,RED_LIGHT_VIOLATION_DETECTION,2024-03-20T22:29:41+05:30,ANPR,AXASignal,Point,"[77.615204, 12.91602]"
4753005,5b778007-797e-4313-b34c-c7c4812365bb,Motorbike,STOP_LINE_VIOLATION,2024-03-20T22:29:40+05:30,ANPR,Jayanagar-4thMain,Point,"[77.580123, 12.930848]"


In [7]:
traffic_df.to_csv("traffic_events.csv", index=False)

In [2]:
traffic_df = pd.read_csv("traffic_events.csv")

# Convert the observationDateTime to a datetime object
traffic_df['observationDateTime'] = pd.to_datetime(traffic_df['observationDateTime'])

# Extract unique locations and times
unique_locations = traffic_df[['location.coordinates', 'observationDateTime']].drop_duplicates()

In [3]:
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

def fetch_weather_data(latitude, longitude, date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    start_date = end_date = date.strftime('%Y-%m-%d')
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "et0_fao_evapotranspiration", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", "is_day", "sunshine_duration"],
        "timezone": "auto"
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    return {
        'hourly': response.Hourly(),
        'latitude': response.Latitude(),
        'longitude': response.Longitude(),
        'elevation': response.Elevation(),
        'timezone': response.Timezone(),
        'timezone_abbreviation': response.TimezoneAbbreviation(),
        'utc_offset_seconds': response.UtcOffsetSeconds()
    }

# Dictionary to store weather data for each location and date
weather_data_dict = {}

for _, row in unique_locations.iterrows():
    coordinates = eval(row['location.coordinates'])
    date = row['observationDateTime']
    lat, lon = coordinates[1], coordinates[0]
    weather_data_dict[(lat, lon, date)] = fetch_weather_data(lat, lon, date)


In [4]:
# Initialize empty lists to hold weather data
temperature_2m = []
relative_humidity_2m = []
dew_point_2m = []
apparent_temperature = []
precipitation = []
rain = []
weather_code = []
pressure_msl = []
surface_pressure = []
cloud_cover = []
et0_fao_evapotranspiration = []
wind_speed_10m = []
wind_speed_100m = []
wind_direction_10m = []
wind_direction_100m = []
wind_gusts_10m = []
is_day = []
sunshine_duration = []

# Initialize empty lists to hold additional features
latitudes = []
longitudes = []
elevations = []
timezones = []
timezone_abbreviations = []
utc_offset_seconds = []

# Iterate over each row in the traffic violations dataset
for _, row in traffic_df.iterrows():
    coordinates = eval(row['location.coordinates'])
    date = row['observationDateTime']
    lat, lon = coordinates[1], coordinates[0]
    weather_data = weather_data_dict.get((lat, lon, date))
    
    if weather_data:
        hourly_data = weather_data['hourly']
        # Find the hour that matches the observation time
        observation_hour = date.hour
        temperature_2m.append(hourly_data.Variables(0).ValuesAsNumpy()[observation_hour])
        relative_humidity_2m.append(hourly_data.Variables(1).ValuesAsNumpy()[observation_hour])
        dew_point_2m.append(hourly_data.Variables(2).ValuesAsNumpy()[observation_hour])
        apparent_temperature.append(hourly_data.Variables(3).ValuesAsNumpy()[observation_hour])
        precipitation.append(hourly_data.Variables(4).ValuesAsNumpy()[observation_hour])
        rain.append(hourly_data.Variables(5).ValuesAsNumpy()[observation_hour])
        weather_code.append(hourly_data.Variables(6).ValuesAsNumpy()[observation_hour])
        pressure_msl.append(hourly_data.Variables(7).ValuesAsNumpy()[observation_hour])
        surface_pressure.append(hourly_data.Variables(8).ValuesAsNumpy()[observation_hour])
        cloud_cover.append(hourly_data.Variables(9).ValuesAsNumpy()[observation_hour])
        et0_fao_evapotranspiration.append(hourly_data.Variables(10).ValuesAsNumpy()[observation_hour])
        wind_speed_10m.append(hourly_data.Variables(11).ValuesAsNumpy()[observation_hour])
        wind_speed_100m.append(hourly_data.Variables(12).ValuesAsNumpy()[observation_hour])
        wind_direction_10m.append(hourly_data.Variables(13).ValuesAsNumpy()[observation_hour])
        wind_direction_100m.append(hourly_data.Variables(14).ValuesAsNumpy()[observation_hour])
        wind_gusts_10m.append(hourly_data.Variables(15).ValuesAsNumpy()[observation_hour])
        is_day.append(hourly_data.Variables(16).ValuesAsNumpy()[observation_hour])
        sunshine_duration.append(hourly_data.Variables(17).ValuesAsNumpy()[observation_hour])
        
        # Add additional features
        latitudes.append(weather_data['latitude'])
        longitudes.append(weather_data['longitude'])
        elevations.append(weather_data['elevation'])
        timezones.append(weather_data['timezone'])
        timezone_abbreviations.append(weather_data['timezone_abbreviation'])
        utc_offset_seconds.append(weather_data['utc_offset_seconds'])
    else:
        temperature_2m.append(None)
        relative_humidity_2m.append(None)
        dew_point_2m.append(None)
        apparent_temperature.append(None)
        precipitation.append(None)
        rain.append(None)
        weather_code.append(None)
        pressure_msl.append(None)
        surface_pressure.append(None)
        cloud_cover.append(None)
        et0_fao_evapotranspiration.append(None)
        wind_speed_10m.append(None)
        wind_speed_100m.append(None)
        wind_direction_10m.append(None)
        wind_direction_100m.append(None)
        wind_gusts_10m.append(None)
        is_day.append(None)
        sunshine_duration.append(None)
        
        latitudes.append(None)
        longitudes.append(None)
        elevations.append(None)
        timezones.append(None)
        timezone_abbreviations.append(None)
        utc_offset_seconds.append(None)

# Add the weather data columns to the traffic violations DataFrame
traffic_df['temperature_2m'] = temperature_2m
traffic_df['relative_humidity_2m'] = relative_humidity_2m
traffic_df['dew_point_2m'] = dew_point_2m
traffic_df['apparent_temperature'] = apparent_temperature
traffic_df['precipitation'] = precipitation
traffic_df['rain'] = rain
traffic_df['weather_code'] = weather_code
traffic_df['pressure_msl'] = pressure_msl
traffic_df['surface_pressure'] = surface_pressure
traffic_df['cloud_cover'] = cloud_cover
traffic_df['et0_fao_evapotranspiration'] = et0_fao_evapotranspiration
traffic_df['wind_speed_10m'] = wind_speed_10m
traffic_df['wind_speed_100m'] = wind_speed_100m
traffic_df['wind_direction_10m'] = wind_direction_10m
traffic_df['wind_direction_100m'] = wind_direction_100m
traffic_df['wind_gusts_10m'] = wind_gusts_10m
traffic_df['is_day'] = is_day
traffic_df['sunshine_duration'] = sunshine_duration

# Add additional features
traffic_df['station.latitude'] = latitudes
traffic_df['station.longitude'] = longitudes
traffic_df['station.elevation'] = elevations
traffic_df['timezone'] = timezones
traffic_df['timezone_abbreviation'] = timezone_abbreviations
traffic_df['utc_offset_seconds'] = utc_offset_seconds

def format_coordinates(lat, lon):
    return [round(lon, 6), round(lat, 6)]

# Apply the function to create the new column
traffic_df['station_coordinates'] = traffic_df.apply(
    lambda row: format_coordinates(row['station.latitude'], row['station.longitude']), axis=1
)

# Save the enriched dataset
traffic_df.to_csv("merged_data.csv", index=False)

# Display the enriched DataFrame
print(traffic_df.head())

                                     id vehicleType  \
0  5b778007-797e-4313-b34c-c7c4812365bb         Car   
1  5b778007-797e-4313-b34c-c7c4812365bb         Car   
2  5b778007-797e-4313-b34c-c7c4812365bb   Motorbike   
3  5b778007-797e-4313-b34c-c7c4812365bb   Motorbike   
4  5b778007-797e-4313-b34c-c7c4812365bb   Motorbike   

                       alertType       observationDateTime cameraUsage  \
0            STOP_LINE_VIOLATION 2024-02-21 13:29:49+05:30        ANPR   
1      LICENCE_PLATE_RECOGNITION 2024-02-21 13:30:00+05:30        ANPR   
2            STOP_LINE_VIOLATION 2024-02-21 13:30:00+05:30        ANPR   
3  RED_LIGHT_VIOLATION_DETECTION 2024-02-21 13:29:59+05:30        ANPR   
4  RED_LIGHT_VIOLATION_DETECTION 2024-02-21 13:10:36+05:30        ANPR   

      junctionName location.type    location.coordinates  temperature_2m  \
0      SampigeRoad         Point  [77.571138, 13.008491]       31.346500   
1      VanivilasJN         Point  [77.576809, 12.948272]       31.385000

In [5]:
print(traffic_df['station_coordinates'])

0          [77.552322, 13.040421]
1          [77.563637, 12.970123]
2          [77.634216, 13.040421]
3          [77.634216, 13.040421]
4          [77.563637, 12.970123]
                    ...          
4753001    [77.645454, 12.970123]
4753002    [77.574928, 12.899824]
4753003    [77.574928, 12.899824]
4753004    [77.574928, 12.899824]
4753005    [77.574928, 12.899824]
Name: station_coordinates, Length: 4753006, dtype: object


In [6]:
unique_coordinates = traffic_df['station_coordinates'].apply(tuple).unique()

In [7]:
print(unique_coordinates)

[(77.552322, 13.040421) (77.563637, 12.970123) (77.634216, 13.040421)
 (77.645454, 12.970123) (77.574928, 12.899824) (77.65667, 12.899824)
 (77.71611, 13.040421) (0.0, -0.035149)]


In [8]:
traffic_df['station_coordinates'] = traffic_df['station_coordinates'].apply(lambda x: tuple(eval(x)) if isinstance(x, str) else tuple(x))
unique_coordinates_elevations = traffic_df[['station_coordinates', 'station.elevation']].drop_duplicates()
print(unique_coordinates_elevations)

           station_coordinates  station.elevation
0       (77.552322, 13.040421)              940.0
1       (77.563637, 12.970123)              914.0
2       (77.634216, 13.040421)              909.0
4       (77.563637, 12.970123)              937.0
5       (77.645454, 12.970123)              912.0
6       (77.563637, 12.970123)              905.0
7       (77.574928, 12.899824)              929.0
11      (77.552322, 13.040421)              942.0
13      (77.645454, 12.970123)              899.0
15      (77.574928, 12.899824)              897.0
19      (77.552322, 13.040421)              922.0
21      (77.634216, 13.040421)              905.0
22      (77.552322, 13.040421)              937.0
29      (77.574928, 12.899824)              926.0
30      (77.563637, 12.970123)              910.0
42      (77.574928, 12.899824)              913.0
51       (77.65667, 12.899824)              897.0
58      (77.563637, 12.970123)              935.0
65      (77.574928, 12.899824)              925.0


In [100]:
import folium

def convert_coordinates(coord):
    if isinstance(coord, str):
        return tuple(map(float, coord.strip('[]').split(', ')))
    elif isinstance(coord, list):
        return tuple(coord)
    else:
        return coord

# Convert 'station_coordinates' to tuples
traffic_df['station_coordinates'] = traffic_df['station_coordinates'].apply(convert_coordinates)

# Extract unique coordinates for traffic violations
unique_coordinates_traffic = traffic_df['station_coordinates'].drop_duplicates()

# Calculate average location for centering the map
average_lat = unique_coordinates_traffic.apply(lambda x: x[1]).mean()  # Assuming latitude is the second element
average_lon = unique_coordinates_traffic.apply(lambda x: x[0]).mean()  # Assuming longitude is the first element

# Create the Folium map
mymap = folium.Map(location=[average_lat, average_lon], zoom_start=12)

# Add markers for traffic violations
for coord in unique_coordinates_traffic:
    folium.Marker(
        location=[coord[1], coord[0]],  # latitude, longitude
        popup=f"Traffic Violation Location<br>Coordinates: [{coord[1]}, {coord[0]}]",
        tooltip="Traffic Violation",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(mymap)


# Save the map as HTML
mymap.save("traffic_violations_map.html")

# Display the map
mymap

In [4]:
import folium

# Create a map centered around the average latitude and longitude of the points
m = folium.Map(location=[12.9716, 77.5946], zoom_start=12)

# Red points
blue_points = [
    (77.571138, 13.008491), (77.63721, 13.022783), (77.593048, 12.989344),
    (77.61448, 12.991661), (77.554162, 12.984416), (77.57275, 13.01463),
    (77.631421, 12.981592), (77.615204, 12.91602), (77.57128, 13.028377),
    (77.643319, 13.009179), (77.59061, 13.010052), (77.553042, 12.998237),
    (77.576809, 12.948272), (77.633651, 12.924831), (77.594711, 12.981865),
    (77.580396, 12.917107), (77.640837, 12.978368), (77.609731, 12.934747),
    (77.580123, 12.930848), (77.591393, 12.953822), (77.704469, 13.017111),
    (77.602438, 12.979232), (77.595183, 12.910289), (77.611378, 12.967623),
    (77.620657, 12.966769), (77.649095, 12.959774), (77.549943, 12.925376),
    (77.646621, 12.978562), (77.57168, 12.960574), (77.578765, 13.006399),
    (77.708519, 13.019476), (0.0, 0.0), (77.607594, 12.944023), (77.586068, 12.928639),
    (77.645087, 12.985709), (77.620982, 12.927565), (77.577328, 12.956281),
    (77.633143, 12.929832), (77.567362, 12.957135), (77.592438, 13.020731),
    (77.638689, 12.960965), (77.641287, 12.970227), (77.622682, 12.942536),
    (77.642785, 13.015277), (77.579713, 12.986486), (77.612338, 12.93438),
    (77.613848, 12.973732), (77.55919, 12.988839), (77.578346, 12.906428),
    (77.546368, 12.997397), (77.580053, 12.943633)
]

# Blue points
red_points = [
    (77.552322, 13.040421), (77.563637, 12.970123), (77.634216, 13.040421),
    (77.645454, 12.970123), (77.574928, 12.899824), (77.65667, 12.899824),
    (77.71611, 13.040421)
]

# Add red markers to the map
for point in blue_points:
    folium.Marker(location=[point[1], point[0]], icon=folium.Icon(color='blue')).add_to(m)

# Add blue markers to the map
for point in red_points:
    folium.Marker(
        location=[point[1], point[0]], 
        icon=folium.Icon(color='red', icon='cloud')
        ).add_to(m)

# Save the map to an HTML file
m.save('map_1.html')

m